In [44]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings , re , traceback
from bs4 import BeautifulSoup
from tqdm import tqdm
import matplotlib.pyplot as plt
import multiprocessing as mp
# from DataProcessing import 
import uszipcode , pickle 
from TempFolder.TempFolder import Temp

warnings.filterwarnings('ignore')

In [70]:
def properties_from_file(file_name):
    make , model , year  = file_name.split('\\')[-1].split('_')
    return make , model , year.split('.')[0]

with open('nogit/path', 'r') as f:
    datapath  = f.read()
path_to_root = datapath + 'SC-carfax\csv\*'
dirs = glob.glob(path_to_root)
data_list = []

# dir_file = dirs[0]
error_ = False
for dir_file in tqdm(dirs):
    make_files = glob.glob(dir_file+'\*')
    for file_path in make_files:
    # file_path = make_files[0]
        try:
            if not error_:
                print('reading the file ',file_path)
                with open(file_path, 'r') as f:
                    fle = f.read()
                if len(fle)>0:
                    soup = BeautifulSoup(fle)
                    print('extracting values')
                    all_values = soup.findAll('li',{'class':'vehicle_details__item'})
                    if len(all_values)>1:
                        values  = (all_values[0]
                                    .text
                                    .split('Original MSRP')[1]
                                    .replace('$','')
                                    .replace(',','')
                                    .split(' - ')
                                    )
                    else:
                        values = [np.nan]
                    make , model , year = properties_from_file(file_path)
                    if len(values) < 2:
                        values.append(values[0])
                    data_blob = dict(make = make, model = model,year = int(year) , Original_MSRP_low = float(values[0]), Original_MSRP_high = float(values[1]))
                    data_list.append(data_blob)
                else:
                    raise Exception('Empty file')
        
        except Exception as e:
            print('error reading ',file_path,'\n',traceback.format_exc())
            if str(e) != 'Empty file':
                error_ = True
res_df = pd.DataFrame(data_list)
res_df.to_csv(datapath + "SC-carfax\processed\MSRP.csv")

  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Crossfire_2006.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Crossfire_2007.html
error reading  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Crossfire_2007.html 
 Traceback (most recent call last):
  File "<ipython-input-70-bea925eb3bd6>", line 42, in <module>
    raise Exception('Empty file')
Exception: Empty file

reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Crossfire_2008.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Grand Voyager_2000.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_LHS_2000.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_LHS_2001.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_Pacifica_2004.html
extracting values
reading the file  c:\data\Datasets\SC-carfax\csv\Chrysler\Chrysler_

In [74]:
res_df

,make,model,year,Original_MSRP_low,Original_MSRP_high
0,Acura,CL,2001,28460.0,30810.0
1,Acura,CL,2002,28530.0,32880.0
2,Acura,CL,2003,28700.0,31050.0
3,Acura,ILX,2013,25900.0,34400.0
4,Acura,ILX,2014,26900.0,34600.0
...,...,...,...,...,...
1475,Oldsmobile,Alero,2004,19110.0,23960.0
1476,Toyota,4Runner,2000,22768.0,37298.0
1477,Toyota,4Runner,2001,26815.0,36585.0
1478,Toyota,4Runner,2002,26845.0,36615.0
